# Java Threads and Concurrency Utilities

## Chapter - 2

In [1]:
Runtime.getRuntime().availableProcessors()

4

### The Problems with Threads

* Race Conditions
* Data Races
* Cached Variables

#### Race Condition

A race condition occurs when the correctness of a computation depends on the relative timing or interleaving of multiple threads by the scheduler.
<pre class="brush:java"> 
if (a == 10.0) 
    b = a / 2.0;
</pre>
There is no problem with this code fragment in a single-threaded context, and there is no problem in a multithreaded context when a and b are local variables. 

However, assume that a and b identify instance or class (static) field variables and that two threads simultaneously access this code, and we are implementing this in multithreaded, this will definately be a disater in this case.

* The code fragment is an example of a common type of race condition that’s known as **check-then-act**, in which a potentially stale observation is used to decide on what to do next. 
* Another type of race condition is **read-modify-write**, in which new state is derived from previous state. The previous state is read, then modified, and finally updated to reflect the modified result via three indivisible operations. However, the combination of these operations isn’t indivisible.

#### Data Races

A race condition is often confused with a data race in which two or more threads (in a single application) access the same memory location concurrently, at least one of the accesses is for writing, and these threads don’t coordinate their accesses to that memory. When these conditions hold, access order is non-deterministic. Different results may be generated from run to run, depending on that order. Consider the following example:

<pre class="brush:java"> 
private static Parser parser;
public static Parser getInstance()
{
if (parser == null) parser = new Parser();
   return parser;
}
</pre>

#### Cached Variables
To boost performance, the compiler, the Java virtual machine (JVM), and the operating system can collaborate to cache a variable in a register or a processor-local cache, rather than rely on main memory. Each thread has its own copy of the variable. When one thread writes to this variable, it’s writing to its copy; other threads are unlikely to see the update in their copies.

#### Synchronizing Access to Critical Sections
You can use synchronization to solve the previous thread problems. Synchronization is a JVM feature that ensures that two or more concurrent threads don’t simultaneously execute a critical section, which is a code section that must be accessed in a serial (one thread at a time) manner.

#### Using Synchronized Methods
A synchronized method includes the synchronized keyword in its header. For example, you can use this keyword to synchronize the former getID() method and overcome its read-modify-write race condition as follows:

In [2]:
// public synchronized int getID()
// {
//    return counter++;
// }

When synchronizing on an instance method, the lock is associated with the object on which the method is called. For example, consider the following ID class:


In [47]:
import java.util.Date;
public class ID {
	private int counter; // initialized to 0 by default

	public synchronized int getID() {
		try {
			Thread.sleep(2000);
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
        System.out.print(new Date()+"\n");
		return counter++;
	}
}

Suppose you specify the following code sequence

The lock is associated with the ID object whose reference is stored in id. If another thread called id.getID() while this method was executing, the other thread would have to wait until the executing thread released the lock.

In [48]:

ID id = new ID();
// id.getID();
Runnable getID = () -> {
	System.out.println(id.getID());
};

Thread t1 = new Thread(getID);
Thread t2 = new Thread(getID);

t1.start();
t2.start();

t1.join();
t2.join();

Wed Jun 17 21:22:06 IST 2020
0
Wed Jun 17 21:22:08 IST 2020
1


In [50]:
public class IDStatic {
	private static int counter; // initialized to 0 by default

	public static synchronized int getID() {
		try {
			Thread.sleep(2000);
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
		System.out.print(new Date() + "\n");
		return counter++;
	}
}

In [51]:
IDStatic id = new IDStatic();
Runnable getID = () -> {
	System.out.println(IDStatic.getID());
};

Thread t1 = new Thread(getID);
Thread t2 = new Thread(getID);

t1.start();
t2.start();

t1.join();
t2.join();

Wed Jun 17 21:25:06 IST 2020
0
Wed Jun 17 21:25:08 IST 2020
1
